In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/kss-Korean-FastSpeech2-Pytorch

/content/drive/.shortcut-targets-by-id/1M6FjTpXPTqlhM1om9afX_m74HOZ5LMEq/kss-Korean-FastSpeech2-Pytorch


In [ ]:
!pip install g2pk scipy jamo librosa tensorboard tgt pyworld

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.9/573.9 kB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 15.0 MB/s eta 0:00:00
  Created wheel for tgt: filename=tgt-1.4.4-py3-none-any.whl size=28902 sha256=104fca467e8b58c7dbf6a50f907fd374bd891ecf0c5469a5e1652726b9132190
  Stored in directory: /root/.cache/pip/wheels/09/e6/aa/821531faeb4e05a65d1c763570e90791467cf0c3f1622dc7e2
  Created wheel for pyworld: filename=pyworld-0.3.4-cp310-cp310-linux_x86_64.whl size=864207 sha256=a0fce766474c63e13ad5d579e20d65110ded573b403a3a35cb44a486e62f7aeb
  S

In [ ]:
import os
from data import kss
import hparams as hp

in_dir = hp.data_path
out_dir = "/content/drive/MyDrive/kss-Korean-FastSpeech2-Pytorch/preprocessed_custom"
meta = hp.meta_name
textgrid_name = hp.textgrid_name

In [ ]:
mel_out_dir = os.path.join(out_dir, "mel")
if not os.path.exists(mel_out_dir):
    os.makedirs(mel_out_dir, exist_ok=True)

ali_out_dir = os.path.join(out_dir, "alignment")
if not os.path.exists(ali_out_dir):
    os.makedirs(ali_out_dir, exist_ok=True)

f0_out_dir = os.path.join(out_dir, "f0")
if not os.path.exists(f0_out_dir):
    os.makedirs(f0_out_dir, exist_ok=True)

energy_out_dir = os.path.join(out_dir, "energy")
if not os.path.exists(energy_out_dir):
    os.makedirs(energy_out_dir, exist_ok=True)

if os.path.isfile(textgrid_name):
    os.system('mv ./{} {}'.format(textgrid_name, out_dir))

if not os.path.exists(os.path.join(out_dir, textgrid_name.replace(".zip", ""))):
    os.system('unzip {} -d {}'.format(os.path.join(out_dir, textgrid_name), out_dir))


In [ ]:
import hparams

hparams.sampling_rate = 22050

In [ ]:
# import numpy as np
# import os
# import tgt
# from scipy.io.wavfile import read
# import pyworld as pw
# import torch
# import audio as Audio
# from utils import get_alignment, standard_norm, remove_outlier, average_by_duration
# import hparams as hp
# from jamo import h2j
# import codecs
# import librosa


# from audio.stft import TacotronSTFT


# from sklearn.preprocessing import StandardScaler

# def prepare_align(in_dir, meta):
#     with open(os.path.join(in_dir, meta), encoding='utf-8') as f:
#         for line in f:
#             parts = line.strip().split('|')
#             basename, text = parts[0], parts[3]

#             basename=basename.replace('.wav','.txt')

#             with open(os.path.join(in_dir,'wavs',basename),'w') as f1:
#                 f1.write(text)t

# # def build_from_path(in_dir, out_dir, meta):
# #     train, val = list(), list()
# #     scalers = [StandardScaler(copy=False) for _ in range(3)]	# scalers for mel, f0, energy
# #     n_frames = 0
# #     with open(os.path.join(in_dir, meta), encoding='utf-8') as f:
# #         for index, line in enumerate(f):
# #             parts = line.strip().split('|')
# #             basename, text = parts[0], parts[3]

# #             ret = process_utterance(in_dir, out_dir, basename, scalers)
# #             if ret is None:
# #                 continue
# #             else:
# #                 info, n = ret
# #             if basename[0] == '1':
# #                 val.append(info)
# #             else:
# #                 train.append(info)

# #             if index % 100 == 0:
# #                 print("Done %d" % index)
# #             n_frames += n
# #     param_list = [np.array([scaler.mean_, scaler.scale_]) for scaler in scalers]
# #     param_name_list = ['mel_stat.npy', 'f0_stat.npy', 'energy_stat.npy']
# #     [np.save(os.path.join(out_dir, param_name), param_list[idx]) for idx, param_name in enumerate(param_name_list)]
# #     return [r for r in train if r is not None], [r for r in val if r is not None]

# _stft = TacotronSTFT(
#     hparams.filter_length, hparams.hop_length, hparams.win_length,
#     hparams.n_mel_channels, hparams.sampling_rate, hparams.mel_fmin,
#     hparams.mel_fmax)

# def process_utterance(in_dir, out_dir, basename, scalers):
#     wav_bak_basename=basename.replace('.wav','')
#     basename = wav_bak_basename[2:]
#     wav_bak_path = os.path.join(in_dir, "wavs_bak", "{}.wav".format(wav_bak_basename))
#     wav_path = os.path.join(in_dir, 'wavs', '{}.wav'.format(basename))

#     tg_path = os.path.join('/content/drive/MyDrive/kss-Korean-FastSpeech2-Pytorch/preprocessed/kss/TextGrid', '{}.TextGrid'.format(basename))

#     print("basename ", basename)

#     # Convert kss data into PCM encoded wavs
#     if not os.path.isfile(wav_path):
#         os.system("ffmpeg -i {} -ac 1 -ar 22050 {}".format(wav_bak_path, wav_path))

#     # Get alignments
#     textgrid = tgt.io.read_textgrid(tg_path)
#     phone, duration, start, end = get_alignment(textgrid.get_tier_by_name('phones'))
#     # text = '{'+ '}{'.join(phone) + '}' # '{A}{B}{$}{C}', $ represents silent phones
#     # text = text.replace('{$}', ' ')    # '{A}{B} {C}'
#     # text = text.replace('}{', ' ')     # '{A B} {C}'

#     if start >= end:
#         return None

#     if not os.path.exists(tg_path):
#         print("tg path not exists", tg_path)
#         return None

#     if not os.path.exists(wav_path):
#       print("wav path not exists")



#     # sampling_rate, data = read(wav_path)
#     # print("sampling rate", sampling_rate)
#     # audio = torch.FloatTensor(data.astpe(np.float32))

#     # if sampling_rate != hparams.sampling_rate:
#     #   print(f"SR doesn't match target! {sampling_rate} {hparams.sampling_rate}")

#     # audio_norm = audio / hparams.max_wav_value
#     # audio_norm = audio_norm.unsqueeze(0)
#     # audio_norm = torch.autograd.Variable(audio_norm, requires_grad=False)
#     # melspec = _stft.mel_spectrogram(audio_norm)
#     # mel_spectrogram = torch.squeeze(melspec, 0)

#     # Read and trim wav files
#     _, wav = read(wav_path)
#     wav = wav[int(hp.sampling_rate*start):int(hp.sampling_rate*end)].astype(np.float32)

#     if  wav.ndim == 2:
#         wav = np.mean(wav, axis=1)
#     # print("wav", wav.shape)

#     # Compute fundamental frequency
#     f0, _ = pw.dio(wav.astype(np.float64), hp.sampling_rate, frame_period=hp.hop_length/hp.sampling_rate*1000)
#     f0 = f0[:sum(duration)]

#     # Compute mel-scale spectrogram and energy
#     mel_spectrogram, energy = Audio.tools.get_mel_from_wav(torch.FloatTensor(wav))
#     mel_spectrogram = mel_spectrogram.numpy().astype(np.float32)[:, :sum(duration)]
#     energy = energy.numpy().astype(np.float32)[:sum(duration)]

#     f0, energy = remove_outlier(f0), remove_outlier(energy)
#     f0, energy = average_by_duration(f0, duration), average_by_duration(energy, duration)

#     if mel_spectrogram.shape[1] >= hp.max_seq_len:
#         return None

#     # Save alignment
#     ali_filename = '{}-ali-{}.npy'.format(hp.dataset, basename)
#     np.save(os.path.join(out_dir, 'alignment', ali_filename), duration, allow_pickle=False)

#     # Save fundamental prequency
#     f0_filename = '{}-f0-{}.npy'.format(hp.dataset, basename)
#     np.save(os.path.join(out_dir, 'f0', f0_filename), f0, allow_pickle=False)

#     # Save energy
#     energy_filename = '{}-energy-{}.npy'.format(hp.dataset, basename)
#     np.save(os.path.join(out_dir, 'energy', energy_filename), energy, allow_pickle=False)

#     # Save spectrogram
#     mel_filename = '{}-mel-{}.npy'.format(hp.dataset, basename)
#     np.save(os.path.join(out_dir, 'mel', mel_filename), mel_spectrogram.T, allow_pickle=False)

#     mel_scaler, f0_scaler, energy_scaler = scalers

#     mel_scaler.partial_fit(mel_spectrogram.T)
#     f0_scaler.partial_fit(f0[f0!=0].reshape(-1, 1))
#     energy_scaler.partial_fit(energy[energy != 0].reshape(-1, 1))

#     return basename, mel_spectrogram.shape[1]



In [ ]:
import numpy as np
import os
import tgt
from scipy.io.wavfile import read
import pyworld as pw
import torch
import audio as Audio
from utils import get_alignment, standard_norm, remove_outlier, average_by_duration
import hparams as hp
from jamo import h2j
import codecs
import librosa
import soundfile as sf
from audio.stft import TacotronSTFT


from sklearn.preprocessing import StandardScaler

def prepare_align(in_dir, meta):
    with open(os.path.join(in_dir, meta), encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('|')
            basename, text = parts[0], parts[3]

            basename=basename.replace('.wav','.txt')

            with open(os.path.join(in_dir,'wavs',basename),'w') as f1:
                f1.write(text)

# def build_from_path(in_dir, out_dir, meta):
#     train, val = list(), list()
#     scalers = [StandardScaler(copy=False) for _ in range(3)]	# scalers for mel, f0, energy
#     n_frames = 0
#     with open(os.path.join(in_dir, meta), encoding='utf-8') as f:
#         for index, line in enumerate(f):
#             parts = line.strip().split('|')
#             basename, text = parts[0], parts[3]

#             ret = process_utterance(in_dir, out_dir, basename, scalers)
#             if ret is None:
#                 continue
#             else:
#                 info, n = ret
#             if basename[0] == '1':
#                 val.append(info)
#             else:
#                 train.append(info)

#             if index % 100 == 0:
#                 print("Done %d" % index)
#             n_frames += n
#     param_list = [np.array([scaler.mean_, scaler.scale_]) for scaler in scalers]
#     param_name_list = ['mel_stat.npy', 'f0_stat.npy', 'energy_stat.npy']
#     [np.save(os.path.join(out_dir, param_name), param_list[idx]) for idx, param_name in enumerate(param_name_list)]
#     return [r for r in train if r is not None], [r for r in val if r is not None]

_stft = TacotronSTFT(
    hparams.filter_length, hparams.hop_length, hparams.win_length,
    hparams.n_mel_channels, hparams.sampling_rate, hparams.mel_fmin,
    hparams.mel_fmax)


def process_utterance(in_dir, out_dir, basename, scalers):
    wav_bak_basename=basename.replace('.wav','')
    basename = wav_bak_basename[2:]
    #wav_bak_path = os.path.join(in_dir, "wavs_bak", "{}.wav".format(wav_bak_basename))
    wav_path = os.path.join(in_dir, 'wavs', '{}.wav'.format(basename))

    tg_path = os.path.join('/content/drive/MyDrive/kss-Korean-FastSpeech2-Pytorch/preprocessed/kss/TextGrid', '{}.TextGrid'.format(basename))

    print("basename ", basename)

    # Convert kss data into PCM encoded wavs
    '''if not os.path.isfile(wav_path):
        os.system("ffmpeg -i {} -ac 1 -ar 22050 {}".format(wav_bak_path, wav_path))'''

    # Get alignments
    textgrid = tgt.io.read_textgrid(tg_path)
    phone, duration, start, end = get_alignment(textgrid.get_tier_by_name('phones'))
    print('duration : ', duration.shape)
    # text = '{'+ '}{'.join(phone) + '}' # '{A}{B}{$}{C}', $ represents silent phones
    # text = text.replace('{$}', ' ')    # '{A}{B} {C}'
    # text = text.replace('}{', ' ')     # '{A B} {C}'

    if start >= end:
        return None

    if not os.path.exists(tg_path):
        print("tg path not exists", tg_path)
        return None

    if not os.path.exists(wav_path):
      print("wav path not exists")



    # sampling_rate, data = read(wav_path)
    # print("sampling rate", sampling_rate)
    # audio = torch.FloatTensor(data.astpe(np.float32))

    # if sampling_rate != hparams.sampling_rate:
    #   print(f"SR doesn't match target! {sampling_rate} {hparams.sampling_rate}")

    # audio_norm = audio / hparams.max_wav_value
    # audio_norm = audio_norm.unsqueeze(0)
    # audio_norm = torch.autograd.Variable(audio_norm, requires_grad=False)
    # melspec = _stft.mel_spectrogram(audio_norm)
    # mel_spectrogram = torch.squeeze(melspec, 0)

    # Read and trim wav files
    sr, wav = read(wav_path)


    if sr != hp.sampling_rate:
      print("sampling rate different", sr)
      y, sr = librosa.load(wav_path, sr=sr)
      wav = librosa.resample(y=y, orig_sr=sr, target_sr=hp.sampling_rate)
      sf.write(wav_path, wav.astype(np.float32), hp.sampling_rate)
      sr, wav = read(wav_path)

    # 0인 부분 잘라내기 텍스트에서 strip같은 역할
    wav = wav[int(hp.sampling_rate*start):int(hp.sampling_rate*end)].astype(np.float32)


    if  wav.ndim == 2:
        wav = np.mean(wav, axis=1)


    # Compute fundamental frequency
    f0, _ = pw.dio(wav.astype(np.float64), hp.sampling_rate, frame_period=hp.hop_length/hp.sampling_rate*1000)
    print('f0 before : ', f0.shape)
    f0 = f0[:sum(duration)]
    print('f0 after : ', f0.shape)

    print("compute mel")

    # Compute mel-scale spectrogram and energy
    print(wav.size())
    mel_spectrogram, energy = Audio.tools.get_mel_from_wav(torch.FloatTensor(wav))
    #print('energy before : ', energy.shape)
    #print('mel : ', mel_spectrogram.shape)
    mel_spectrogram = mel_spectrogram.numpy().astype(np.float32)[:, :sum(duration)]
    energy = energy.numpy().astype(np.float32)[:sum(duration)]
    #print('energy after : ', energy.shape)

    print("computed mel")

    f0, energy = remove_outlier(f0), remove_outlier(energy)
    #print('final1 f0, energy : ', f0.shape, energy.shape)
    f0, energy = average_by_duration(f0, duration), average_by_duration(energy, duration)
    #print('final2 f0, energy : ', f0.shape, energy.shape)

    print("f0")

    if mel_spectrogram.shape[1] >= hp.max_seq_len:
        return None

    # Save alignment
    ali_filename = '{}-ali-{}.npy'.format(hp.dataset, basename)
    np.save(os.path.join(out_dir, 'alignment', ali_filename), duration, allow_pickle=False)

    # Save fundamental prequency
    f0_filename = '{}-f0-{}.npy'.format(hp.dataset, basename)
    np.save(os.path.join(out_dir, 'f0', f0_filename), f0, allow_pickle=False)

    # Save energy
    energy_filename = '{}-energy-{}.npy'.format(hp.dataset, basename)
    np.save(os.path.join(out_dir, 'energy', energy_filename), energy, allow_pickle=False)

    # Save spectrogram
    mel_filename = '{}-mel-{}.npy'.format(hp.dataset, basename)
    np.save(os.path.join(out_dir, 'mel', mel_filename), mel_spectrogram.T, allow_pickle=False)

    print("saved")

    mel_scaler, f0_scaler, energy_scaler = scalers

    mel_scaler.partial_fit(mel_spectrogram.T)
    f0_scaler.partial_fit(f0[f0!=0].reshape(-1, 1))
    energy_scaler.partial_fit(energy[energy != 0].reshape(-1, 1))

    return basename, mel_spectrogram.shape[1]



In [ ]:
def build_from_path(in_dir, out_dir, meta):
    train, val = list(), list()
    scalers = [StandardScaler(copy=False) for _ in range(3)]	# scalers for mel, f0, energy
    n_frames = 0
    with open(os.path.join(in_dir, meta), encoding='utf-8') as f:
        for index, line in enumerate(f):
            parts = line.strip().split('|')
            basename, text = parts[0], parts[3]

            ret = process_utterance(in_dir, out_dir, basename, scalers)

            if index % 100 == 0:
                print("Done %d" % index)

    param_list = [np.array([scaler.mean_, scaler.scale_]) for scaler in scalers]
    param_name_list = ['mel_stat.npy', 'f0_stat.npy', 'energy_stat.npy']
    [np.save(os.path.join(out_dir, param_name), param_list[idx]) for idx, param_name in enumerate(param_name_list) if not os.isfile(os.path.join(out_dir, param_name))]
    return [r for r in train if r is not None], [r for r in val if r is not None]


In [ ]:
out_dir = '/content/drive/MyDrive'

In [ ]:
build_from_path(in_dir, out_dir, meta)

basename  1_0000
compute mel
computed mel
f0
saved
Done 0
basename  3_2540


<ipython-input-7-cc9f93827bb0>:109: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, wav = read(wav_path)


sampling rate different 44100
compute mel
computed mel
f0
saved
basename  3_2541


<ipython-input-7-cc9f93827bb0>:109: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, wav = read(wav_path)


sampling rate different 44100
compute mel
computed mel
f0
saved
basename  3_2542


...

sampling rate different 44100
compute mel
computed mel
f0
saved
basename  4_5630


<ipython-input-7-cc9f93827bb0>:109: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, wav = read(wav_path)


sampling rate different 44100
compute mel
computed mel
f0
saved
basename  4_5631
sampling rate different 44100
compute mel
computed mel
f0
saved


<ipython-input-7-cc9f93827bb0>:109: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, wav = read(wav_path)


([], [])

In [ ]:
# from audio.stft import TacotronSTFT
# from audio.audio_processing import griffin_lim
# import hparams
# import soundfile as sf

# _stft = TacotronSTFT(
#     hparams.filter_length, hparams.hop_length, hparams.win_length,
#     hparams.n_mel_channels, hparams.sampling_rate, hparams.mel_fmin,
#     hparams.mel_fmax)


# mel = mel_filename = '{}-mel-{}.npy'.format(hp.dataset, "1_0003")
# mel_spectrogram = np.load(os.path.join(out_dir, 'mel', mel_filename))

# mel_torch = torch.tensor(mel_spectrogram)


# mel = torch.stack([mel_torch])
# mel_decompress = _stft.spectral_de_normalizea(mel)
# mel_decompress = mel_decompress.data.cpu()
# spec_from_mel_scaling = 1000
# spec_from_mel = torch.mm(mel_decompress[0].to(torch.float32), _stft.mel_basis.to(torch.float32))
# spec_from_mel = spec_from_mel.transpose(0, 1).unsqueeze(0)
# spec_from_mel = spec_from_mel * spec_from_mel_scaling

# print("spec_from_mel size ", spec_from_mel.size())

# audio = griffin_lim(magnitudes=torch.autograd.Variable(spec_from_mel[:, :, :-1]), stft_fn=_stft.stft_fn)

# audio = audio.squeeze()
# audio_np = audio.cpu().numpy()
# # audio_np = audio[0].data.cpu().numpy()
# sampling_rate = hparams.sampling_rate

# # 출력 파일 경로 설정 (파일 이름을 적절하게 수정)
# output_file_path = f"/content/drive/MyDrive/kss-Korean-FastSpeech2-Pytorch/preprocessed_custom/mel.wav"

# # 오디오를 WAV 파일로 저장
# sf.write(output_file_path, audio_np.astype(np.float32), sampling_rate)


spec_from_mel size  torch.Size([1, 513, 191])


In [ ]:
out_dir = "/content/drive/MyDrive/kss-Korean-FastSpeech2-Pytorch/preprocessed/kss"

In [ ]:
## scaler ##

scalers = [StandardScaler(copy=False) for _ in range(3)]	# scalers for mel, f0, energy
n_frames = 0
with open(os.path.join(in_dir, meta), encoding='utf-8') as f:
    for index, line in enumerate(f):
        parts = line.strip().split('|')
        basename, text = parts[0], parts[3]

        basename = basename.replace('.wav','')[2:]

        # Save alignment
        ali_filename = '{}-ali-{}.npy'.format(hp.dataset, basename)
        duration = np.load(os.path.join(out_dir, 'alignment', ali_filename))


        if not os.path.exists(os.path.join(out_dir, 'alignment', ali_filename)):
          print("path not exists", basename)

        # Save fundamental prequency
        f0_filename = '{}-f0-{}.npy'.format(hp.dataset, basename)
        f0 = np.load(os.path.join(out_dir, 'f0', f0_filename))

        # Save energy
        energy_filename = '{}-energy-{}.npy'.format(hp.dataset, basename)
        energy = np.load(os.path.join(out_dir, 'energy', energy_filename))

        # Save spectrogram
        mel_filename = '{}-mel-{}.npy'.format(hp.dataset, basename)
        mel_spectrogram = np.load(os.path.join(out_dir, 'mel', mel_filename))

        mel_scaler, f0_scaler, energy_scaler = scalers

        mel_scaler.partial_fit(mel_spectrogram)
        f0_scaler.partial_fit(f0[f0!=0].reshape(-1, 1))
        energy_scaler.partial_fit(energy[energy != 0].reshape(-1, 1))

        if index % 100 == 0:
          print(index)


param_list = [np.array([scaler.mean_, scaler.scale_]) for scaler in scalers]
param_name_list = ['mel_stat.npy', 'f0_stat.npy', 'energy_stat.npy']
[np.save(os.path.join(out_dir, param_name), param_list[idx]) for idx, param_name in enumerate(param_name_list)]

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800


[None, None, None]

In [ ]:
import shutil
import os

# Source directory where the files are currently located
source_directory = "/content/drive/MyDrive/kss-Korean-FastSpeech2-Pytorch/preprocessed_custom/energy"
a = os.listdir(source_directory)
print(a)
# Destination directory where you want to move the files
destination_directory = "/content/drive/MyDrive/kss-Korean-FastSpeech2-Pytorch/preprocessed/kss/energy"

# Ensure the destination directory exists; create it if it doesn't
os.makedirs(destination_directory, exist_ok=True)

# Move each file to the destination directory
for file_name in a:
    if not file_name.endswith('.npy'):
      continue
    source_path = os.path.join(source_directory, file_name)
    destination_path = os.path.join(destination_directory, file_name)

    try:
        shutil.move(source_path, destination_path)
        print(f"Moved '{file_name}' to '{destination_directory}'")
    except FileNotFoundError:
        print(f"File '{file_name}' not found in '{source_directory}'")
    except Exception as e:
        print(f"An error occurred while moving '{file_name}': {str(e)}")


[]


In [ ]:
# import numpy as np
# import os


# def check_difference(prev_path, curr_path):
#   missing_files = 0
#   total_files = 0
#   file_list = os.listdir(prev_path)
#   for file in file_list:
#     total_files += 1
#     prev_data = np.load(os.path.join(prev_path, file))
#     try:
#       curr_data = np.load(os.path.join(curr_path, file))
#     except:
#       missing_files += 1
#       continue
#     if not np.array_equal(prev_data, curr_data):
#       print("basename :", basename)
#       print("prev_data :", prev_data)
#       print("curr_data :", curr_data)
#       print("missing file :", missing_files)
#       print("total file :", total_files)
#       return False
#   print("missing file :", missing_files)
#   print("total file :", total_files)
#   return True

In [ ]:
# prev_path = "/content/drive/MyDrive/kss-Korean-FastSpeech2-Pytorch/preprocessed/kss/alignment"
# curr_path = "/content/drive/MyDrive/kss-Korean-FastSpeech2-Pytorch/preprocessed_custom/alignment"

In [ ]:
# print(check_difference(prev_path, curr_path))

missing file : 5885
total file : 12854
True


In [ ]:
# prev_path = "/content/drive/MyDrive/kss-Korean-FastSpeech2-Pytorch/preprocessed/kss/energy"
# curr_path = "/content/drive/MyDrive/kss-Korean-FastSpeech2-Pytorch/preprocessed_custom/energy"

In [ ]:
# print(check_difference(prev_path, curr_path))

prev_data : [5.4987027e-05 2.7661158e-03 2.7899991e-03 2.1020623e-03 1.0501664e-03
 6.0556841e-04 1.3759843e-03 1.3104999e-03 4.6984991e-04 2.2345772e-03
 1.1464901e-03 7.9451251e-04 2.3272245e-03 1.9095867e-03 2.1372889e-03
 2.3207278e-03 3.6311633e-04 1.6249005e-03 1.4280112e-03 4.4404165e-04
 8.3464163e-04 6.0641370e-04 2.5897773e-03 7.2644040e-04 1.0797662e-03
 6.6622766e-04 1.5012606e-03 4.6041994e-05 1.5923090e-04 1.7827936e-03
 1.5731916e-03 1.2324542e-03 1.4838056e-03 1.3085578e-03 1.3241168e-03
 8.6356573e-05 5.6284538e-04 2.3720552e-04]
curr_data : [5.4995748e-05 2.7661861e-03 2.7899891e-03 2.1021503e-03 1.0501868e-03
 6.0582015e-04 1.3759914e-03 1.3104834e-03 4.7005221e-04 2.2346529e-03
 1.1465445e-03 7.9451315e-04 2.3272936e-03 1.9095936e-03 2.1372817e-03
 2.3207688e-03 3.6312762e-04 1.6250022e-03 1.4280127e-03 4.4422186e-04
 8.3474780e-04 6.0640986e-04 2.5898293e-03 7.2665873e-04 1.0798041e-03
 6.6624238e-04 1.5012810e-03 4.6044748e-05 1.5923506e-04 1.7828121e-03
 1.573191

In [ ]:
# prev_path = "/content/drive/MyDrive/kss-Korean-FastSpeech2-Pytorch/preprocessed/kss/f0"
# curr_path = "/content/drive/MyDrive/kss-Korean-FastSpeech2-Pytorch/preprocessed_custom/f0"

In [ ]:
# print(check_difference(prev_path, curr_path))

KeyboardInterrupt: ignored

In [ ]:
# prev_path = "/content/drive/MyDrive/kss-Korean-FastSpeech2-Pytorch/preprocessed/kss/mel"
# curr_path = "/content/drive/MyDrive/kss-Korean-FastSpeech2-Pytorch/preprocessed_custom/mel"

In [ ]:
# print(check_difference(prev_path, curr_path))

KeyboardInterrupt: ignored